In [19]:
import numpy as np
import pandas as pd
import matplotlib
#%matplotlib qt5
matplotlib.use("Qt5Agg")

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation



In [20]:
import numpy as np
import pandas as pd

import functools
import itertools

import random
import tqdm

import multiprocessing
import threading


# One bee in R^n
class Bee(object):

    def __init__(self, min_bounds, max_bounds, function, sigma, c1, c2):

        # self.file = open("{}.log".format(random.randint(1, 10000000000)), "w")

        self.position = np.zeros(min_bounds.shape)
        for i in range(len(min_bounds)):
            self.position[i] = np.random.uniform( min_bounds[i],
                                                  max_bounds[i], 1 ) 

        self.function = function
        self.velocity = np.random.normal( 0, sigma, len(min_bounds) )
        self.c1 = c1
        self.c2 = c2
        
        self.sigma = sigma
        
        self.local_minimum = np.copy(self.position)
        self.local_minimum_value = self.function( self.local_minimum )
        
        self.min_bounds = min_bounds[:]
        self.max_bounds = max_bounds[:]

        #self.counter = 0

    def calculate_next_velocity(self, global_minimum, w):
        
        r1, r2 = np.random.uniform(0, 1, 2)

        to_local =  (self.local_minimum - self.position)
        #to_local /= np.linalg.norm(to_local, ord=2) + 0.01
        #to_local *= 10
        
        to_global = global_minimum - self.position
        #
        #to_global /= np.linalg.norm(to_global, ord=2) + 0.01
        #to_global *= 10
        
        self.velocity = self.velocity * w + self.c1 * r1 * to_local + \
                    self.c2 * r2 * to_global + \
                    np.random.normal(0, np.linalg.norm(self.velocity) / 3 + 1, 2)

        #if (self.position == global_minimum).all():
        #    self.counter += 1
            
        #if self.counter == 5:
        #    self.counter = 0
        #    self.position = np.zeros(min_bounds.shape)
        #    for i in range(len(min_bounds)):
        #        self.position[i] = np.random.uniform( min_bounds[i],
        #                                              max_bounds[i], 1 ) 

        #    self.velocity = np.random.normal( 0, self.sigma, len(min_bounds) )
        

    def move(self):

        self.position += self.velocity
        
        self.position = np.minimum(self.position, self.max_bounds)
        self.position = np.maximum(self.position, self.min_bounds)
        
        fc = self.function(self.position)

        if fc < self.local_minimum_value:
            self.local_minimum_value = fc
            self.local_minimum = np.copy(self.position)

        return (self.local_minimum, self.local_minimum_value)



In [50]:

class BeeColony():

    def __init__(self, n_bees, min_bounds, max_bounds,
                 function, n_iter, weight):

        # The magic constants here
        self.w = weight

        self.bees = [ Bee(min_bounds, max_bounds, function, 10, 0.2, 0.05)
                     for i in range(n_bees) ]
        self.n_iter = n_iter
        self.min_bounds = min_bounds[:]
        self.max_bounds = max_bounds[:]

        
        self.global_minimum = np.zeros(min_bounds.shape)
        #for i in range(len(min_bounds)):
        #    self.global_minimum[i] = np.random.uniform( min_bounds[i], max_bounds[i], 1 )
            
        self.global_minimum_value = function(self.global_minimum)

        self.function = function
        
        
    def run(self):

        coords = []
        
        for it in list(range(self.n_iter)):

            #print('Iteration:', it)
            local = []

            for bee in self.bees:
                bee.calculate_next_velocity(self.global_minimum, self.w)

                retval = bee.move()
                local.append(retval)

            local.sort(key=lambda x: x[1])

            if local[0][1] < self.global_minimum_value:
                self.global_minimum, self.global_minimum_value = local[0][0], local[0][1]

            tmp = [ np.copy(bee.position) for bee in self.bees ]
            #print(len(tmp))
                    
            coords.append(tmp)
            
            # print('-' * 60)
            #input()
        return np.array(coords)

In [51]:
p1 = np.random.uniform(-0.5, -0.1, 1)
p2 = np.random.uniform(0.1, 0.9, 1)
p3, p4, p5 = np.random.uniform(0.1, 0.5, 3)
p6, p7, p8 = np.random.uniform(100, 400, 3)


def function1(x):
    result = 418.9829 * len(x)

    for i in range(len(x)):
        result += ( -(abs(x[i]) + 1) ** (p1) * np.sin( np.abs( x[i] ) ** p2  ) * x[i] ) 

    n = np.linalg.norm(x, ord=2)
    return  -result * (1 + np.sin(x[1] / p6) * p3 + np.sin(x[0] / p7) * p4 + np.cos(n / p8) * p5  )

def function(x):
    return function1(x) 


def get_lattice(l1, l2):
    xx, yy = np.meshgrid(l1, l2)
    zz = np.zeros(xx.shape)
    for i in range(zz.shape[0]):
        for j in range(zz.shape[1]):
            zz[i, j] = function( [ xx[i, j], yy[i, j] ]  )
    return zz

f = function

min_bounds = np.array([ -1950, -1950 ])
max_bounds = np.array([  1950,  1950 ])



#crd = b.run()

In [52]:
l = np.linspace(-2000, 2000, 401)
xx, yy = np.meshgrid(l, l)
zz = get_lattice(l, l)

In [57]:
b = BeeColony(10, min_bounds, max_bounds, f, n_iter=1, weight=1 )

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)
ax.contour(xx, yy, zz, linewidths=2)
ax.contourf(xx, yy, zz, alpha=0.6)
ax.grid(color='black', linewidth=1)


crd = b.run()
tg = ax.scatter(crd[0, :, 0], crd[0, :, 1], animated=True, s=10, color='yellow')
gm = ax.scatter(b.global_minimum[0], b.global_minimum[1], s=200, marker='*', color='red')    

lc = np.array([ bee.local_minimum for bee in b.bees ])    
l1 = ax.scatter(lc[:, 0], lc[:, 1], color='orange', marker='x')

def update(frame):
    #if frame % 10 == 0:
    #    print(frame)
    crd = b.run()
    tg.set_offsets(crd[0])
    gm.set_offsets( [b.global_minimum] )
    lc = np.array([ bee.local_minimum for bee in b.bees ])    
    l1.set_offsets(lc)
    
      
    b.w *= 0.9999
    #print(f(b.global_minimum) - b.global_minimum_value)
    
    return tg, gm, l1

ani = FuncAnimation(fig, update, frames=1500, blit=True, interval=100, repeat=False)
#ani.save('line.mp4', writer='ffmpeg')
#print('Here')
plt.show()

In [54]:
import sys
import numpy as np

from vispy import app, scene
from vispy import color
from vispy import visuals
from vispy.gloo import gl


In [58]:

canvas = scene.SceneCanvas(keys='interactive', bgcolor='#0a0a2a', size=(1000, 700))
view = canvas.central_widget.add_view()
view.camera = scene.TurntableCamera(up='z', fov=60, distance=60)

#from function import get_lattice

l = np.array( np.linspace(-2000, 2000, 401) )
xx, yy = np.meshgrid(l, l)
z = get_lattice(l, l)

y = np.copy(z)
y -= np.min(y)
y /= np.max(y)
y = - y + 1

c = color.get_colormap("hsl").map(y).reshape(z.shape + (-1,))
c = c.flatten().tolist()
c = list(map(lambda x,y,z,w:(x,y,z,w), c[0::4],c[1::4],c[2::4],c[3::4]))

#print(c.shape)
print(z.shape)

pt1 = scene.visuals.SurfacePlot(x=l, y=l, z=z, parent=view.scene)
pt1.mesh_data.set_vertex_colors(c)

pt1.transform = scene.transforms.MatrixTransform()
pt1.transform.translate([0, 0, -np.average(z)-10])
pt1.transform.scale([1., 1., 1/3.])
pt1.transform.scale([1/60, 1/60, 1/60])

#view.add(p2)

view.add(pt1)

# p1._update_data()  # cheating.
# cf = scene.filters.ZColormapFilter('fire', zrange=(z.max(), z.min()))
# p1.attach(cf)


xax = scene.Axis(pos=[[-0.5, -0.5], [0.5, -0.5]], tick_direction=(0, -1),
                 font_size=25, axis_color='red', tick_color='red',
                 text_color='red', parent=view.scene)

xax.transform = scene.STTransform(translate=(0, 0, -0.2))

yax = scene.Axis(pos=[[-0.5, -0.5], [-0.5, 0.5]], tick_direction=(-1, 0),
                font_size=25, axis_color='red', tick_color='red',
                text_color='red', parent=view.scene)

yax.transform = scene.STTransform(translate=(0, 0, -0.2))

zax = scene.Axis(pos=[[-0.5, -0.5, 0], [-0.5, -0.5, 1]], tick_direction=(0, -1),
                font_size=25, axis_color='red', tick_color='red',
                text_color='red', parent=view.scene)

#zax.transform = scene.STTransform(translate=(0, 0, -0.2))

# Add a 3D axis to keep us oriented
# axis = scene.visuals.XYZAxis(parent=view.scene)

if __name__ == '__main__':
    canvas.show()
    if sys.flags.interactive == 0:
        app.run()


(401, 401)


  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/alexander/.local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/alexander/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/alexander/.local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/alexander/.local/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 112, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.6/asyncio/base_events.py", line 421, in run_forever
    self._run_once()
  File "/usr/lib/python3.6/asyncio/base_events.py", line 1426, in _run_once
    handle._run()
  File "/usr/lib/python3.6/asyncio/events.py", line 127, in _run
    self._call